# Yelp Data Merging

In [1]:
# Copied and pasted from my imports from project 3, 
# delete if we dont need(Aerika)

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.feature_extraction import stop_words

from sklearn.pipeline import Pipeline

from sklearn.model_selection import train_test_split, GridSearchCV

from sklearn.linear_model import LogisticRegression

from sklearn.naive_bayes import MultinomialNB, ComplementNB, GaussianNB, BernoulliNB

from sklearn.metrics import confusion_matrix, accuracy_score, roc_curve, auc

from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from nltk.stem import SnowballStemmer
from nltk.tokenize import ToktokTokenizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.corpus import stopwords 

import regex as re

from scipy.stats import ttest_ind

In [2]:
file_paths = []
for file in os.listdir('../data'):
    if 'businesses2019' in file:
        file_paths.append('../data/'+file)

In [3]:
master_df = {
    'id': [],
    'latitude': [],
    'longitude': [],
    'price': [],
    'review_count': [],
    'rating': [],
    'zip_code': [],
    'city': [],
    'alias': [],
    'category': [],
}

master_df = pd.DataFrame(master_df)

In [4]:
for path in file_paths:
    
    master_df = pd.concat([master_df, pd.read_csv(path)])

master_df.head()

C:\Users\m6zha\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  This is separate from the ipykernel package so we can avoid doing imports until


,alias,category,city,id,latitude,longitude,price,rating,review_count,zip_code
0,jack-in-the-box-lebec-3,"['hotdogs', 'breakfast_brunch', 'burgers']",Lebec,g9QI0mMCq4D07jxATL5qRA,34.818608,-118.882458,$,2.0,77.0,93243
1,los-pinos-mexican-restaurant-lebec-2,['mexican'],Lebec,qRsFaV-cukBGL2nlJoktYw,34.818766,-118.883621,$$,4.0,268.0,93243
2,subway-restaurants-lebec,['sandwiches'],Lebec,UP21YTmGG5uDMXlPQVpskQ,34.818628,-118.884271,$,3.5,11.0,93243
3,quiznos-lebec,"['hotdogs', 'sandwiches']",Lebec,2zcJfPZ8lDq-HU9n56gBuw,34.818439,-118.885094,$,3.5,5.0,93243
4,china-red-house-chinese-restaurant-lebec,['chinese'],Lebec,uNzapwAHu-WaTtrN41p-UA,34.818343,-118.885659,$,4.0,42.0,93243


In [5]:
# Seeing shape of dataframe BEFORE dropping duplicates 

master_df.shape

(102673, 10)

In [6]:
# Dropping duplicates

master_df = master_df.sort_values(['alias',
                       'id', 'latitude', 'longitude', 'review_count'],
                      ascending = False).drop_duplicates(subset = ['id'],
                                                         keep ='first')

In [7]:
# Shape of df AFTER dropping duplicates

master_df.shape

(28884, 10)

In [8]:
# Resetting master_df index

master_df.reset_index(inplace=True)
master_df.drop(columns='index', inplace=True)

# NaNs in master_df

In [9]:
master_df.isna().sum()

alias             1
category          1
city              1
id                1
latitude         17
longitude        17
price             1
rating            1
review_count      1
zip_code        124
dtype: int64

In [10]:
# Dropping rows w/ prices that aren't $, $$, $$$, or $$$$.

data_drop_df = master_df.loc[(master_df['price'] != '$') & (master_df['price'] != '$$') & (master_df['price'] != '$$$') & (master_df['price'] != '$$$$')]
drop_indexes = list(data_drop_df.index)

master_df.drop(index=drop_indexes, inplace = True)
master_df.shape


(28712, 10)

In [11]:
# Businesses that don't have latitude/zipcodes = Food Trucks, Caterers --> dropped

food_trucks = master_df.loc[(master_df['latitude'].isna()) | (master_df['zip_code'].isna())].index.tolist()

master_df.drop(index = food_trucks, inplace = True)
master_df.shape


(28595, 10)

In [12]:
master_df.isna().sum()

alias           0
category        0
city            0
id              0
latitude        0
longitude       0
price           0
rating          0
review_count    0
zip_code        0
dtype: int64

# Replacing \\$ in price to numbers

In [13]:
master_df['price'] = master_df['price'].map({'$$$$':4, '$$$':3, '$$':2,'$':1})

In [14]:
master_df.head(2)

,alias,category,city,id,latitude,longitude,price,rating,review_count,zip_code
0,重庆特色小面-chongqing-special-noodles-san-gabriel-3,"['noodles', 'chinese']",San Gabriel,mm6l24khOfXhKqn4pKVojA,34.103070,-118.091840,1,4.0,204.0,91776
1,重庆小面-best-noodle-house-rosemead-3,"['noodles', 'szechuan']",Rosemead,DgXxBgUEvARu45x7RSgYRw,34.081182,-118.066226,1,4.0,141.0,91770


# Dropping Rows 
- convert zip_codes from float -> int -> str
- zipcodes that don't start with '9'

In [15]:
master_df = master_df.reset_index(drop=True)

In [16]:
master_df['zip_code'] = master_df['zip_code'].astype(int).astype(str)

In [17]:
notin_la = []
for i in range(len(master_df['zip_code'])):
    if master_df['zip_code'][i].startswith('9') == False:
        notin_la.append(master_df['zip_code'].index[i])

In [18]:
notin_la

[941,
 1716,
 3009,
 4514,
 6610,
 8017,
 9538,
 9823,
 9827,
 9838,
 9842,
 9848,
 9852,
 9858,
 9860,
 9869,
 9871,
 9883,
 9892,
 9914,
 9935,
 9936,
 9953,
 9957,
 9959,
 9964,
 9966,
 9967,
 9968,
 9983,
 9985,
 9988,
 9996,
 14656,
 14872,
 15115,
 16420,
 18577,
 23914,
 25246,
 25384,
 26870]

In [19]:
master_df.shape

(28595, 10)

In [20]:
master_df = master_df.drop(index=notin_la).reset_index(drop=True)

In [21]:
master_df.shape

(28553, 10)

# Removing rows that aren't around the same latitude/longitude

In [22]:
master_df.describe()

,latitude,longitude,price,rating,review_count
count,28553.000000,28553.000000,28553.000000,28553.000000,28553.000000
mean,34.050451,-118.242524,1.482226,3.585455,242.046895
std,0.241629,0.386881,0.578135,0.812601,420.873217
min,33.167953,-149.429066,1.000000,0.000000,0.000000
25%,33.950983,-118.384980,1.000000,3.000000,37.000000
50%,34.052071,-118.263988,1.000000,3.500000,107.000000
75%,34.131160,-118.123680,2.000000,4.000000,280.000000
max,61.581385,-84.414552,4.000000,5.000000,16662.000000


In [23]:
master_df = master_df.drop(index=master_df.loc[(master_df['longitude'] > -117) |
                                               (master_df['latitude'] > 35) | 
                                               (master_df['longitude'] < -120)].index).reset_index(drop=True)

In [24]:
master_df.shape

(28545, 10)

# Feature Engineering

## Save the cleaned data to a CSV

In [25]:
import os

In [26]:
os.getcwd()

'C:\\Users\\m6zha\\Desktop\\github\\Predicting-Neighborhood-Affluence-with-Yelp\\Code'

In [28]:
# Don't uncomment this unless you actually want to create a new csv

master_df.to_csv('../data/master.csv')